# Experiment Tutorial

In [1]:
from shrinkbench.experiment import PruningExperiment
import os
import torch
import pickle
import glob

The `DATAPATH` and `WEIGHTSPATH` environment variables are used to tell the framework where to look for datasets and pretrained weights respectively.

In [2]:
os.environ['DATAPATH'] = '/nfs3/data/aniruddha:/nfs3/data/aniruddha/ULP/tiny_imagenet/Attacked_Data/Triggers_11_20'
os.environ['WEIGHTSPATH'] = '/nfs3/data/aniruddha/ULP/tiny_imagenet/poisoned_models/Triggers_11_20'

Code snippet to prune a TinyImageNet model.

In [4]:
# Choose which layer to prune
module_list= ['layer4.0.conv1']

# load ULPs and classifier
# ANIRUDDHA - load ULPs to calculate parameter gradients
N=10
ULPs, W, b=pickle.load(open('/nfs3/code/aniruddha/ULP/tiny_imagenet/Results/universal_image_ResNet18_mod_N{}.pkl'.format(N),'rb'))

# Weight pruning using product of parameter value and ULP gradient
strategy = 'LayerMagGrad'
# Dummy compression value. Keep this as 1. ULP importance score with Gaussian distribution clipping implement in override function           
c = 1                               

# Load list of poisoned models
poisoned_model_list = sorted(glob.glob('/nfs3/data/aniruddha/ULP/tiny_imagenet/poisoned_models/Triggers_11_20/*.pt'))
# Load poisoned model meta information
meta = pickle.load(open('/nfs3/data/aniruddha/ULP/Webpage/tiny-imagenet/poisoned_models_Triggers_11_20_meta.pkl', 'rb'))
# Number of models to run experiment on
NUM_MODELS = 2

for i in range(NUM_MODELS):
    exp = PruningExperiment(dataset='TinyImageNet',         # Choose dataset
                            model='resnet18_mod',           # Choose model architecture
                            pretrained=False,               
                            resume=poisoned_model_list[i],  # Model checkpoint
                            strategy=strategy,              # Pruning strategy
                            compression=c,                  # This argument used by default in Shrinkbench library
                            train_kwargs={'epochs':1},      # Number of epochs of finetuning after pruning
                            source=meta[i][1],              # Backdoor source class
                            target=meta[i][2],              # Backdoor target class
                            module_list=module_list,        # List of prunable modules.
                            poisoned_root=os.path.basename(meta[i][3]), # Root directory of poisoned data.
                            override_fraction=True,         # This overrides compression argument to use backdoor gaussian distribution fit.
                            ULP_data=[ULPs, W, b]           # ULP images and classifier weight and bias.
                            )
    exp.run()

# results saved in results/TinyImageNet 